In [2]:
import pandas as pd 
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import fonts
from openpyxl.chart import BarChart, reference
import string 

In [3]:
excel_file = pd.read_csv('/home/dataengineering/Desktop/digitalskola/dataset/sales_2021.csv')
excel_file[['Gender','Product line','Total']]
print(excel_file[['Gender','Product line','Total']])

     Gender            Product line      Total
0    Female       Health and beauty   548.9715
1    Female  Electronic accessories    80.2200
2      Male      Home and lifestyle   340.5255
3      Male       Health and beauty   489.0480
4      Male       Sports and travel   634.3785
..      ...                     ...        ...
995    Male       Health and beauty    42.3675
996  Female      Home and lifestyle  1022.4900
997    Male      Food and beverages    33.4320
998    Male      Home and lifestyle    69.1110
999  Female     Fashion accessories   649.2990

[1000 rows x 3 columns]


In [4]:
#Pivot table
report_table = excel_file.pivot_table(index='Gender', columns='Product line', values='Total',aggfunc='sum').round(0)

print(report_table)

Product line  Electronic accessories  Fashion accessories  Food and beverages  \
Gender                                                                          
Female                       27102.0              30437.0             33171.0   
Male                         27236.0              23868.0             22974.0   

Product line  Health and beauty  Home and lifestyle  Sports and travel  
Gender                                                                  
Female                  18561.0             30037.0            28575.0  
Male                    30633.0             23825.0            26548.0  


In [5]:
report_table.to_excel('report_2021.xlsx', sheet_name='Report',startrow=4)

In [6]:
wb = load_workbook('report_2021.xlsx')
min_column = wb.active.min_column
max_column = wb.active.max_column
min_row = wb.active.min_row
max_row = wb.active.max_row

print("Min Column" '',min_column)
print("Max Column" '',max_column)
print("Min Row"'',min_row)
print("Max Row"'',max_row)

Min Column 1
Max Column 7
Min Row 5
Max Row 7


In [14]:
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import fonts
from openpyxl.chart import BarChart, Reference

wb = load_workbook('report_2021.xlsx')
sheet = wb['Report']
# formatting the report
sheet['A1'] = 'Sales Report'
sheet['A2'] = '2021'
sheet['A1'].font = Font('Arial', bold=True, size=20)
sheet['A2'].font = Font('Arial', bold=True, size=12)

#adding a chart
barchart = BarChart()

data = Reference(sheet, min_col=min_column+1, max_col=max_column, min_row=min_row, max_row=max_row) #including headers
categories = Reference(sheet, min_col=min_column, max_col=min_column, min_row=min_row+1, max_row=max_row) #not including headers
   
barchart.add_data(data, titles_from_data=True)
barchart.set_categories(categories)

#location chart
sheet.add_chart(barchart, "B12") 
sheet.title = 'Sales by Product line'
barchart = 2
wb.save('report_wchart.xlsx')


In [12]:
import string
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import Font

wb = load_workbook('report_2021.xlsx')
sheet = wb['Report']

# applying formulas
# first create alphabet list as references for cells
alphabet = list(string.ascii_uppercase)
excel_alphabet = alphabet[0:max_column] 
print(excel_alphabet)
#note: Python lists start on 0 -> A=0, B=1, C=2. 
#note2 the [a:b] takes b-a elements

#Applying sum 
for i in excel_alphabet:
         if i!='A':
             sheet[f'{i}{max_row+1}'] = f'=SUM({i}{min_row+1}:{i}{max_row})'
             sheet[f'{i}{max_row+1}'].style = 'Currency'
sheet[f'{excel_alphabet[0]}{max_row+1}'] = 'Total'
sheet[f'{excel_alphabet[0]}{max_row+1}'].font = Font(bold=True)
wb.save('report_2021.xlsx')

['A', 'B', 'C', 'D', 'E', 'F', 'G']


In [15]:
def automate_excel(file_name):
    """The file name should have the following structure: sales_month.xlsx"""
    
    # read excel file
    excel_file = pd.read_excel(file_name)

    
    # make pivot table
    report_table = excel_file.pivot_table(index='Gender', columns='Product line', values='Total', aggfunc='sum').round(0)
    
    # splitting the month and extension from the file name
    month_and_extension = file_name.split('_')[1]
   
    # send the report table to excel file
    report_table.to_excel(f'report_{month_and_extension}', sheet_name='Report', startrow=4)
    
    # loading workbook and selecting sheet
    wb = load_workbook(f'report_{month_and_extension}')
    sheet = wb['Report']
    
    # cell references (original spreadsheet)
    min_column = wb.active.min_column
    max_column = wb.active.max_column
    min_row = wb.active.min_row
    max_row = wb.active.max_row
    
    # adding a chart
    barchart = BarChart()
    data = Reference(sheet, min_col=min_column+1, max_col=max_column, min_row=min_row, max_row=max_row) #including headers
    categories = Reference(sheet, min_col=min_column, max_col=min_column, min_row=min_row+1, max_row=max_row) #not including headers
    barchart.add_data(data, titles_from_data=True)
    barchart.set_categories(categories)
    sheet.add_chart(barchart, "B12") #location chart
    barchart.title = 'Sales by Product line'
    barchart.style = 2 #choose the chart style
    
    # applying formulas
    # first create alphabet list as references for cells
    alphabet = list(string.ascii_uppercase)
    excel_alphabet = alphabet[0:max_column] #note: Python lists start on 0 -> A=0, B=1, C=2. #note2 the [a:b] takes b-a elements
    
    # sum in columns B-G
    for i in excel_alphabet:
        if i!='A':
            sheet[f'{i}{max_row+1}'] = f'=SUM({i}{min_row+1}:{i}{max_row})'
            sheet[f'{i}{max_row+1}'].style = 'Currency'
    sheet[f'{excel_alphabet[0]}{max_row+1}'] = 'Total'
    
    # getting month name
    month_name = month_and_extension.split('.')[0]
    
    # formatting the report
    sheet['A1'] = 'Sales Report'
    sheet['A2'] = month_name.title()
    sheet['A1'].font = Font('Arial', bold=True, size=20)
    sheet['A2'].font = Font('Arial', bold=True, size=10)
    wb.save(f'report_{month_and_extension}')
    

In [19]:
automate_excel("sales_2021.xlsx")